# Simple enzymatic mechanism
fse--> wpe

In [1]:
from biocrnpyler import *
# same color scheme as the others for presentation
import bokeh.io
import bokeh.plotting
from bokeh.layouts import row
import colorcet
bokeh.io.output_notebook()

Loading BokehJS ...

In [2]:
def check_type(item, material_type_str): 
    if isinstance(item, Species):
        item_ret = item
    elif isinstance(item, str):
        item_ret = Species(name = item, material_type = material_type_str)
    elif isinstance(item, Component) and item.get_species() != None:
        item_ret = item.get_species()
    else:
        raise ValueError( item, "parameter must be a string, a Component with defined get_species(), or a chemical_reaction_network.species")
    return item_ret
class FuelMichaelisMenten(Mechanism):
    def __init__(self, name, type = 'catalysis', **keywords):
        
        Mechanism.__init__(self = self, name = name, mechanism_type = type, **keywords)

    def update_species(self, enzyme, fuel_list, substrate_list, product_list, waste_list): 
        
        self.enzyme = check_type(enzyme, 'enzyme')
        
        species = [self.enzyme]
        comp1_list = [self.enzyme]
        comp2_list = [self.enzyme]
        
        for f in fuel_list:
            species.append(f)
            comp1_list.append(f)

        for s in substrate_list:
            species.append(s)
            comp1_list.append(s)
            
        for p in product_list:
            species.append(p)
            comp2_list.append(p)
            
        for w in waste_list:
            species.append(w)
            comp2_list.append(w)
               
        
        species += [ComplexSpecies(comp1_list)]
        species += [ComplexSpecies(comp2_list)]
        return species
    
    def update_reactions(self, enzyme, fuel_list, substrate_list, product_list, waste_list, k_bf, k_uf, T, component = None,
                        part_id = None): 
        
        # Reverse binding rates
        k_br = 0.1*k_bf
        k_ur = 0.1*k_uf

        self.enzyme = check_type(enzyme, 'enzyme')
        
        # Define input lists
        comp1_list = [self.enzyme]
        comp2_list = [self.enzyme]
        
        for f in fuel_list:
            comp1_list.append(f)
            
        for s in substrate_list:
            comp1_list.append(s)

        for p in product_list:
            comp2_list.append(p)
            
        for w in waste_list:
            comp2_list.append(w)
        
        # Define Complexes
        comp1 = ComplexSpecies(comp1_list)
        comp2 = ComplexSpecies(comp2_list)
        
        # Get K_Cat
        k_eff=2.08*(10**10)
        R=8.31446261
        e=2.718
        Gcat = 70
        Ginact = 90
        Km = 1e-05
        # multiple substrate k_cat:
        
#         # original string ayush gave me
#         #k_cat= '((' + str(k_eff) + '*'+ str(T) + ')*' + str(e) + '**(-' + str(Gcat)+ '/(' + str(R) + '*' + str(T)+ ')))'
        
#         # string from his notebook
#         k_cat =  "((" + str(k_eff) +"*"+ str(T) + ")*" + str(e) + "^(-" + str(Gcat)+ "/(" + str(R) + "*" + str(T)+"))*" + "(" + str(self.enzyme)
#         for s in substrate_list:
#             k_cat += '*' + str(s)
        
#         k_cat += ")" + "/" +"(" + str(Km)
        
#         for s in substrate_list:
#             k_cat += '+' + str(s)
        
#         k_cat += ")" + ")"
#        # + str(Sub) + ")" + "/" +"(" + str(Km) + "+" + str(Sub) + ")" + ")"

#         print(k_cat)
        # actual value
    
#         k_cat = ((k_eff * T)*e**(-Gcat/(R*T)))
#         k_inact = ((k_eff * T)*e**(-Ginact/(R*T)))
        
        k_cat = ((k_eff * T)*e**(-Gcat/(R*T))) * 60 * 60
        k_inact = ((k_eff * T)*e**(-Ginact/(R*T))) * 60 * 60

       # k_inact= '((' + str(k_eff) + '*' + str(T) + ')*' + str(e) + '**(-' + str(Ginact)+ '/(' + str(R) + '*' + str(T)+ ')))'
        
        #print(k_inact)
        #k_cat = 3.4740433199013525e-10
        # Define Reactions
       # binding_rxn = Reaction(inputs = comp1_list, outputs=[comp1], k = k_bf, k_rev = k_br)
        #cat_rxn = Reaction(inputs = [comp1], outputs = [comp2],  k=1,propensity_type= "general", propensity_params= { "rate": k_cat})
        cat_rxn = Reaction(inputs = comp1_list, outputs = comp2_list,  k=k_cat)


       # unbinding_rxn = Reaction(inputs = [comp2], outputs = comp2_list, k=k_uf, k_rev = k_ur)

       # death_rxn = Reaction(inputs = [enzyme], outputs = [], k=1, propensity_type = 'general', propensity_params = {'rate':k_inact})
        death_rxn = Reaction(inputs = [enzyme], outputs = [], k=k_inact)

        
        
        return [cat_rxn, death_rxn]
    #return item_ret

In [3]:
class Enzyme(Component): # k_cat = 36000
    def __init__(self, enzyme_name, substrate, fuel, product, waste, k_bf, k_uf, T, **keywords):
      
        # ENZYME NAME
        self.enzyme = check_type(enzyme_name, 'enzyme')
    
        # SUBSTRATE
        self.substrate_list = []
        for s in substrate:
            self.substrate_list.append(self.set_species(s, material_type = 'molecule'))
            
        # FUEL
        self.fuel_list = []
        for f in fuel:
            self.fuel_list.append(self.set_species(f, material_type ='metabolite'))
        
        
        # PRODUCT
        self.product_list = []
        for p in product:
            self.product_list.append(self.set_species(p, material_type = 'molecule'))
          
            
        # WASTE
        self.waste_list = []
        for w in waste:
            self.waste_list.append(self.set_species(w, material_type = 'metabolite'))
        
        
        self.k_bf = k_bf
        self.k_uf = k_uf
        self.T = T
      
        Component.__init__(self = self, name = enzyme_name, **keywords)
        
    def update_species(self):
        mech_cat = self.mechanisms['catalysis']

        return mech_cat.update_species(self.enzyme, self.fuel_list, self.substrate_list, self.product_list, self.waste_list) 
                                                                                           
    
    def update_reactions(self):
        mech_cat = self.mechanisms['catalysis']

        return mech_cat.update_reactions(self.enzyme, self.fuel_list, self.substrate_list, self.product_list, self.waste_list,self.k_bf,
                                         self.k_uf, self.T,component = None,  part_id = None)
    

In [4]:
class EnergyTxTl(Mixture):
    def __init__(self, name="",**keywords): 
        

        mech_cat = FuelMichaelisMenten('catalysis')
        
        default_mechanisms = {
            mech_cat.mechanism_type:mech_cat
        }
        
        Mixture.__init__(self, name = name, default_mechanisms=default_mechanisms, **keywords)     

In [5]:
def try_temps(T, t_max, p, p1, i):
    # Define rate constants

    k_bf = 15 * 60 * 60
    k_uf = 15 * 60 * 60
    #T = 300
    # Define reaction for each enzyme
    E1_hex = Enzyme(enzyme_name = "hex", substrate = ['glucose'],
                fuel = ['atp'],product = ['g6p'], waste = ['adp'], k_bf = k_bf , k_uf = k_uf, T = T)

    E2_pgi = Enzyme(enzyme_name = 'pgi', substrate = ['g6p'], fuel = [],
               product = ['f6p'], waste = [], k_bf = k_bf , k_uf = k_uf, T = T)

    E3_pfk = Enzyme(enzyme_name = 'pfk', substrate = ['f6p'], fuel = ['atp'], product = ['f16p'],
               waste = ['adp'], k_bf = k_bf , k_uf = k_uf, T = T)

    E4_ald_tpi = Enzyme(enzyme_name ='ald_tpi' , substrate = ['f16p'], fuel = [], product = ['g3p', 'g3p'], 
                waste = [], k_bf = k_bf , k_uf = k_uf, T = T )

    E5_gapN = Enzyme(enzyme_name ='gapN' , substrate = ['g3p', 'g3p'], fuel = ['nadp', 'nadp'], product = ['3pg', '3pg'], 
                waste = ['nadph', 'nadph'], k_bf = k_bf , k_uf = k_uf, T = T)

    E6_gapM6 = Enzyme(enzyme_name ='gapM6' , substrate = ['g3p', 'g3p'], fuel = ['pi'], product = ['13bpg'], 
                waste = [],k_bf = k_bf , k_uf = k_uf, T = T)

    E7_pgk = Enzyme(enzyme_name = 'pgk', substrate = ['13bpg'], fuel = ['adp'], product = ['3pg', '3pg'], 
                waste = ['atp'], k_bf = k_bf , k_uf = k_uf, T = T)

    E8_pgm = Enzyme(enzyme_name ='pgm' , substrate = ['3pg', '3pg'], fuel = [], product = ['2pg', '2pg'], 
                waste = [], k_bf = k_bf , k_uf = k_uf, T = T)

    E9_eno = Enzyme(enzyme_name ='eno' , substrate = ['2pg', '2pg'], fuel = [], product = ['pep', 'pep'], 
                waste = [],k_bf = k_bf , k_uf = k_uf, T = T)

    E10_pyk = Enzyme(enzyme_name = 'pyk', substrate = ['pep', 'pep'], fuel = ['adp', 'adp'], product = ['pyruvate', 'pyruvate'], 
                waste = ['atp', 'atp'], k_bf = k_bf , k_uf = k_uf, T = T) # irreversible

    E11_alsS = Enzyme(enzyme_name = 'alsS', substrate = ['pyruvate', 'pyruvate'], fuel = [], product = ['acetolac'], 
                waste = [], k_bf = k_bf , k_uf = k_uf, T = T) # irreversible

    E12_IlvC = Enzyme(enzyme_name = 'IlvC', substrate = ['acetolac'], fuel = ['nadph'], product = ['23dih3mebut'], 
                waste = ['nadp'], k_bf = k_bf , k_uf = k_uf, T = T)

    E13_IlvD = Enzyme(enzyme_name ='IlvD' , substrate = ['23dih3mebut'], fuel = [], product = ['3me2oxo'], 
                waste = [],k_bf = k_bf , k_uf = k_uf, T = T)

    E14_kivD = Enzyme(enzyme_name ='kivD' , substrate = ['3me2oxo'], fuel = [], product = ['isobutanal'], 
                waste = [], k_bf = k_bf , k_uf = k_uf, T = T) # irreversible

    E15_yahk = Enzyme(enzyme_name = 'yahk', substrate = ['isobutanal'], fuel = ['nadph'], product = ['isobutanol'],
                      waste = ['nadp'], k_bf = k_bf , k_uf = k_uf, T = T)

    # Define ATP Leak Only Enzyme
    # k_bf_atp = 1
    # k_uf_atp = 1
    E16_all_other_atp = Enzyme(enzyme_name = 'atp_synthase', substrate = [], fuel = ['atp'], 
                               product = [], waste = ['adp', 'pi'], k_bf = k_bf, k_uf = k_uf, T = T)

    # Define the Mixture for Rheostat + ATP Leak
    # myMixture = EnergyTxTl(components = [E1_hex,E2_pgi,E3_pfk, E4_ald_tpi, E5_gapN, E6_gapM6, E7_pgk, E8_pgm, E9_eno, E10_pyk, 
    #                                     E11_alsS, E12_IlvC, E13_IlvD, E14_kivD, E15_yahk, E16_all_other_atp])

    # Define the Mixture for Rheostat + no ATP Leak
    myMixture = EnergyTxTl(components = [E1_hex,E2_pgi,E3_pfk, E4_ald_tpi, E5_gapN, E6_gapM6, E7_pgk, E8_pgm, E9_eno, E10_pyk, 
                                        E11_alsS, E12_IlvC, E13_IlvD, E14_kivD, E15_yahk])

    CRN = myMixture.compile_crn()

    # Define Mixture for ATP Leak only
    myMixture_atp = EnergyTxTl(components = [E16_all_other_atp])
    CRN_atp = myMixture_atp.compile_crn()
    
    
    # CRN.write_sbml_file("CRN.sbml")
    # CRN_atp.write_sbml_file("CRN_atp.sbml")

    # Define timepoints
    timepoints = np.linspace(0,t_max,70)

    # Define initial conditions for Rheostat + ATP Leak Model
    e = 0.15
    e4 = 3.5
    atp = 30
    x0 = {'molecule_glucose':30,
          'metabolite_atp': atp,
         'metabolite_nadp':atp,
          "enzyme_hex":e,
          'enzyme_pgi':e,
          'enzyme_pfk':e,
          'enzyme_ald_tpi':e,
          'enzyme_gapN':e, 
          'enzyme_gapM6':e,
          'enzyme_pgk':e,
          'enzyme_pgm':e,
          'enzyme_eno':e,
          'enzyme_pyk':e,
          'enzyme_alsS':e,
          'enzyme_IlvC':e,
          'enzyme_IlvD':e,
          'enzyme_kivD':e,
          'enzyme_yahk':e,
        #  "enzyme_atp_synthase":e4
    }

    #Define initial conditions for ATP Leak Model Only
    x0_atp = {'metabolite_atp':atp,
              "enzyme_atp_synthase":e4,
    }

    # Perform simulations
    re = CRN.simulate_with_bioscrape(timepoints, initial_condition_dict = x0, full_output = 1)
    re_atp = CRN_atp.simulate_with_bioscrape(timepoints, initial_condition_dict = x0_atp)


    
    
    #colors=['#1b9e77','#d95f02','#7570b3','#e7298a','#66a61e','#e6ab02','#a6761d']
    colors = colorcet.b_glasbey_category10
    #colors = ['#8dd3c7','#ffffb3','#bebada','#fb8072','#80b1d3','#fdb462','#b3de69','#fccde5','#d9d9d9','#bc80bd','#ccebc5','#ffed6f']
    
    p.line(timepoints*60*60, re['metabolite_atp'], legend_label = f'ATP for {T} K'.format(T), line_width = 2, color = colors[i])
    p1.line(timepoints*60*60, re['enzyme_hex'], legend_label = f'enzyme_hex for {T} K', line_width = 2, color = colors[i])


    # First plot glucose and isobutanol
    return p



In [7]:
temp_list = [1, 100, 200, 310,340]
p = bokeh.plotting.figure(width = 450, height = 250, 
                         x_axis_label = 'time (sec)',
                         y_axis_label = 'concentration (mM)',
                         title = 'ATP for different Temperatures for simple mechanism')

p1 = bokeh.plotting.figure(width = 450, height = 250, 
                         x_axis_label = 'time (sec)',
                         y_axis_label = 'concentration (mM)',
                         title = 'enzyme_hex at different temperatures')

for i,temp in enumerate(temp_list):
    
    try_temps(temp, 0.01, p,p1, i)

p.legend.click_policy = 'hide'
bokeh.io.show(row(p, p1))